In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,901 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Pac

In [ ]:
!pip install requests beautifulsoup4

In [ ]:
# Potrebni importi
!pip install selenium

!apt-get install chromium-browser
!apt-get install chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 37.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-browser is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
chromium-browser set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

static_url = "https://tip.ba/"
driver.get(static_url)

print("Učitana stranica:", driver.title)

Učitana stranica: Početna - TIP.ba


In [ ]:
import requests
from bs4 import BeautifulSoup
import time

putanja_fajla = "/content/drive/MyDrive/clanci/clanci_pdf.txt"
static_url = "https://tip.ba"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

kategorije_info = {
    "teme/tuzla": {"naziv": "Tuzla - TK", "stranica_max": 402}
    "teme/drustvo": {"naziv": "Društvo", "stranica_max": 653},
    "teme/bih": {"naziv": "BiH", "stranica_max": 494}
    "teme/politika": {"naziv": "Politika", "stranica_max": 319},
    "teme/regija-svijet": {"naziv": "Regija - Svijet", "stranica_max": 200}
    "teme/crna-hronika": {"naziv": "Crna Hronika", "stranica_max": 179},
    "teme/kolumne": {"naziv": "Kolumne", "stranica_max": 6},
    "teme/sport": {"naziv": "Sport", "stranica_max": 176}
}

def get_soup(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return BeautifulSoup(response.text, "html.parser")
    return None

with open(putanja_fajla, "w", encoding="utf-8") as f:
    for kategorija, info in kategorije_info.items():
        naziv_kategorije = info["naziv"]
        broj_stranica = info["stranica_max"]

        for stranica in range(1, broj_stranica + 1):
            if stranica == 1:
                url = f"{static_url}/{kategorija}/"
            else:
                url = f"{static_url}/{kategorija}/page/{stranica}/"

            print(f"\n=== Otvaranje: {url} ===")
            soup = get_soup(url)
            if not soup:
                print("× Nije moguće učitati stranicu.")
                continue

            blokovi = soup.select("div.flex.flex-row.sm\\:block.hover-img h3 > a")
            clanak_linkovi = list(set(a["href"] for a in blokovi if a.get("href")))

            if not clanak_linkovi:
                print("× Nema više članaka, kraj kategorije.")
                break

            for link in clanak_linkovi:
                try:
                    print(f"→ Otvaram link: {link}")
                    clanak_soup = get_soup(link)
                    if not clanak_soup:
                        print("× Greška pri otvaranju članka.")
                        continue

                    # Naslov
                    naslov_el = clanak_soup.select_one("h2.text-3xl.font-bold")
                    naslov = naslov_el.get_text(strip=True) if naslov_el else "N/A"

                    # Datum
                    datum_el = clanak_soup.select_one("div.my-4.text-sm time")
                    datum = datum_el.get_text(strip=True) if datum_el else "N/A"

                    # Autor
                    paragrafi = clanak_soup.select("p")
                    autor = "N/A"
                    for p in paragrafi:
                        tekst = p.get_text()
                        if "Autor:" in tekst or "Autorica:" in tekst:
                            if "Autorica:" in tekst:
                                autor = tekst.split("Autorica:")[1].split("/")[0].strip()
                            elif "Autor:" in tekst:
                                autor = tekst.split("Autor:")[1].split("/")[0].strip()
                            break

                    # Tekst članka
                    tekst_paragrafi = clanak_soup.select("div.leading-relaxed.main-post.pb-4 p")
                    tekst = "\n".join(p.get_text(strip=True) for p in tekst_paragrafi if p.get_text(strip=True))

                    # Upis u fajl
                    f.write(f"Novina: Tip.ba\n")
                    f.write(f"Datum: {datum}\n")
                    f.write(f"Kategorija: {naziv_kategorije}\n")
                    f.write(f"Nadnaslov: N/A\n")
                    f.write(f"Naslov: {naslov}\n")
                    f.write(f"Podnaslov: N/A\n")
                    f.write(f"Strana: {stranica}\n")
                    f.write(f"Autor(i): {autor}\n")
                    f.write("Tekst:\n")
                    f.write(f"{tekst}\n")
                    f.write("=" * 80 + "\n")

                    print(f"[✓] Sačuvan članak: {link}")
                    time.sleep(0.5)

                except Exception as e:
                    print(f"× Greška kod članka: {e}")
                    continue

print("\n✅ Završeno. Podaci su zapisani u 'clanci_pdf.txt'.")


Streaming output truncated to the last 5000 lines.
→ Otvaram link: https://tip.ba/2024/01/03/foto-skandalozna-poruka-pripadnik-sudske-policije-bih-uz-sliku-ratnog-zlocinca-mladica-cestitao-novu-godinu/
[✓] Sačuvan članak: https://tip.ba/2024/01/03/foto-skandalozna-poruka-pripadnik-sudske-policije-bih-uz-sliku-ratnog-zlocinca-mladica-cestitao-novu-godinu/
→ Otvaram link: https://tip.ba/2024/01/04/mladi-ljubusak-izumio-chonchu-pametnu-posudu-za-cvijece/
[✓] Sačuvan članak: https://tip.ba/2024/01/04/mladi-ljubusak-izumio-chonchu-pametnu-posudu-za-cvijece/
→ Otvaram link: https://tip.ba/2024/01/03/sve-je-vise-ljudi-koji-jedini-obrok-dobiju-u-javnoj-kuhinji/
[✓] Sačuvan članak: https://tip.ba/2024/01/03/sve-je-vise-ljudi-koji-jedini-obrok-dobiju-u-javnoj-kuhinji/
→ Otvaram link: https://tip.ba/2024/01/03/ljekari-u-engleskoj-pocinju-najduzi-strajk-ikad/
[✓] Sačuvan članak: https://tip.ba/2024/01/03/ljekari-u-engleskoj-pocinju-najduzi-strajk-ikad/
→ Otvaram link: https://tip.ba/2024/01/03/nep